In [ ]:
#  AdjacencyGraph

mutable struct AdjacencyGraph
    commonGenes::Set{AbstractGene}
    adjA::Vector{Adjacency}
    adjB::Vector{Adjacency}
    cycles::Int
    ab_paths::Int
    a_runs::Int
    b_runs::Int
    run_potential::Int
    indel_potential::Int
end

function AdjacencyGraph(A::Genome, B::Genome)
    gene_set_a = Set(Iterators.flatten(A.data))
    gene_set_b = Set(Iterators.flatten(B.data))
    commonGenes = intersect(gene_set_a, gene_set_b)
    if Telomere() in commonGenes
        delete!(commonGenes, Telomere())
    end
    gene_set_a = setdiff(gene_set_a, commonGenes)
    gene_set_b = setdiff(gene_set_b, commonGenes)

    adjA = Adjacency[]
    adjB = Adjacency[]
    adj = [GeneEnd(Telomere()), GeneEnd(Telomere())]
    adjacencies = [adjA, adjB]
    reference_A = Dict{GeneEnd, Int}(GeneEnd(Telomere(), true) => 0)
    reference_B = Dict{GeneEnd, Int}(GeneEnd(Telomere(), true) => 0)
    references = [reference_A, reference_B]

    for (i, genome) in enumerate([A, B])
        adjacency_length = 0
        for chromosome in genome.data
            index = 0
            adj_index = 0
            current_gene = chromosome.data[index]
            chromosome_genes = Set(chromosome.data)
            adjacency_length += length(intersect(commonGenes, chromosome_Genes))
            if chromosome.data[1] isa Telomere
                adjacency_length += 1
            end
            while length(adjacencies[i]) < adjacency_length
                label = AbstractGene[]
                if current_gene isa Telomere
                    adj[1] = GeneEnd(Telomere(), true)
                else
                    adj[1] = GeneEnd(current_gene, !current_gene.reverse)
                end
                next_gene = ifelse(index >= length(chromosome.data) - 1, chromosome.data[1], chromosome.data[index + 1])
                while next_gene ∉ commonGene
                    if next_gene isa Telomere
                        break
                    end
                    push!(label, next_gene)
                    index += 1
                    next_gene = ifelse(index >= length(chromosome.data) - 1, chromosome.data[1], chromosome.data[index + 1])
                end
                if next_gene isa Telomere
                    adj[2] = GeneEnd(Telomere(), true)
                else
                    adj[2] = GeneEnd(next_gene, next_gene.reverse)
                end
                push!(adjacencies[i], Adjacency(adj[1], adj[2], label))
                if adj[1].gene != Telomere()
                    references[i][adj[1]] = length(adjacencies[i])
                end
                if adj[2].gene != Telomere()
                    references[i][adj[2]] = length(adjacencies[i])
                end
                current_gene = next_gene
                index += 1
                adj_index += 1
            end
        end
    end

    # traversing adj graph 
    to_visit_a_index = Set(1:length(adjA))
    to_visit_b_index = Set(1:length(adjB))
    visited_a_index = Set{Int}()
    visited_b_index = Set{Int}()
    cycles = 0
    ab_paths = 0
    a_runs = 0
    on_a_run = false
    b_runs = 0
    on_b_run = false

    while !isempty(to_visit_a_index)
        current_adj_index = pop!(to_visit_a_index)
        push!(visited_a_index, current_adj_index)
        left_gene = adjA[current_adj_index].left_end_gene
        right_gene = adjA[current_adj_index].right_end_gene
        if !isempty(adjA[current_adj_index].label)
            a_runs += 1
            on_a_run = true
        end
        paths_end_on_a = [true, true]
        for (i, current_gene) in enumerate([left_gene, right_gene])
            a_side = true
            next_adj_index = reference_B[current_gene]
            if next_adj_index in visited_b_index
                continue
            end
            while next_adj_index != nothing
                current_adj_index = next_adj_index
                a_side = !a_side
                adj_side = a_side ? adjA : adjB
                current_adj = adj_side[current_adj_index]
                reference_side = a_side ? reference_B : reference_A
                if a_side
                    push!(visited_a_index, current_adj_index)
                    delete!(to_visit_a_index, current_adj_index)
                else
                    push!(visited_b_index, current_adj_index)
                    delete!(to_visit_b_index, current_adj_index)
                end
                if !isempty(current_adj.label)
                    if !a_side && on_a_run
                        on_b_run = true
                        on_a_run = false
                        b_runs += 1
                    elseif !a_side && !on_b_run
                        on_b_run = true
                        b_runs += 1
                    elseif a_side && on_b_run
                        on_a_run = true
                        on_b_run = false
                        a_runs += 1
                    elseif a_side && !on_a_run
                        on_a_run = true
                        a_runs += 1
                    end
                end
                current_gene = current_gene == current_adj.left_end_gene ? current_adj.right_end_gene : current_adj.left_end_gene
                next_adj_index = reference_side[current_gene]
                if next_adj_index == nothing
                    paths_end_on_a[i] = a_side
                    break
                elseif (a_side && next_adj_index in visited_b_index) || (!a_side && next_adj_index in visited_a_index)
                    cycles += 1
                    break
                end
            end
        end
        if paths_end_on_a[1] != paths_end_on_a[2]
            ab_paths += 1
        end
    end

    run_potential = a_runs + b_runs
    indel_potential = run_potential > 0 ? (run_potential + 1) ÷ 2 + ((run_potential ÷ 2) % 2) : 0

    return AdjacencyGraph(commonGenes, adjA, adjB, cycles, ab_paths, a_runs, b_runs, run_potential, indel_potential)
end



In [ ]:
# pseudo for algo 1 - construction of adj graph 

# src_location_to_adj = Dict{Int, Vector{GeneEnd}}() 
# src_gene_to_location = Dict{Gene, Vector{Int}}()

# target_location_to_adj = Dict{Int, Vector{GeneEnd}}() 
# target_gene_to_location = Dict{Gene, Vector{Int}}()

# for each adj {p, q} in src genome 
    # create edge connecting m, {p, q} and vertex of genome B that contains p 
    # create edge connecting {p, q} and vertex of genome B that contains q


# for each telomere {p} in src genome 
    # create edge connecting {p} and the vertex of target that contains p


In [ ]:
# pseudo for algo 2 - dcj operations and distance

# retain count 

# for each adj {p, q} in genome B 
#     let u be element of genome A that contains p 
#     let v be element of genome A that contains q 
#     if u != v 
#         replace u and v in A by {p, q} and (u\{p}) U (v\{q})
#         count++ 

# for each telomere {p} in genome B 
#     let u be element of genome A that contains p 
#     if u is an adjacency then 
#         replace u in A by {p} and (u\{p})
#         count++ 

In [ ]:
  # converting char to idx in alphabet 
    # Int.(['a','c']) .- Int('a') .+ 1

In [ ]:
"""function combine_ge(adj_list::Vector{Adjacency}, u::GeneEnd, u_idx::Int, v::GeneEnd, v_idx::Int, otherge::Bool)
    if u_idx < v_idx 
        u_lt_v = true 
    else 
        u_lt_v = false 
    end 

    u_adj = src_adj_list[u_idx]
    v_adj = src_adj_list[v_idx]
    
    
    if otherge  # (u\{p}) U (v\{q}) 
        new_u = other_adjacency_end(u, u_adj)
        new_v = other_adjacency_end(v, v_adj)
    else  # {u, v} retaining original order 

    end 

    if !u_lt_v 
        return Adjacency(new_u, new_v)
    else 
        return Adjacency(new_v, new_u)
    end

end """

In [20]:
function next_permutation(perm)
  """
  Finds the next permutation of a list in lexicographical order.

  Args:
      perm: A list of integers representing a permutation.

  Returns:
      The next permutation of the input list, or None if there is no next permutation.
  """
  n = length(perm)
  i = n - 2
  while i >= 1 && perm[i] >= perm[i + 1]
    i -= 1
  end
  if i == 0
    return nothing
  end
  j = n - 1
  while j > i && perm[j] <= perm[i]
    j -= 1
  end
  perm[i], perm[j] = perm[j], perm[i]
  reverse!(perm[i + 1:end])
  return perm
end

function nth_permutation(n, elements)
  """
  Generates the nth permutation of a list of elements in lexicographical order.

  Args:
      n: The index of the desired permutation (1-based).
      elements: A list of unique integers representing the elements to permute.

  Returns:
      The nth permutation of the input elements.
  """
  perm = elements
  n -= 1
  while n > 0
    perm = next_permutation(perm)
    if perm == nothing
      break
    end
    n -= 1
  end
  return perm
end

# Example usage
elements = [1, 2, 3]
first_permutation = nth_permutation(1, elements)
second_permutation = nth_permutation(2, elements)

println(first_permutation)  # Output: [1, 2, 3]
println(second_permutation) # Output: [1, 3, 2]


LoadError: BoundsError: attempt to access 3-element Vector{Int64} at index [0]